# RUNNARE SOLO SU COLAB

In [1]:
# 1. Installa Conda su Colab
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [1]:
# 2. Verifica Conda e Clona il Repo
import condacolab
condacolab.check()

# Clona il repository ufficiale (se non esiste già)
import os
if not os.path.exists("EgoVLP"):
    !git clone https://github.com/showlab/EgoVLP.git

%cd EgoVLP

# Installa le dipendenze richieste tramite Conda
# Nota: PyTorch è già stato parzialmente installato nel passaggio precedente, qui ci assicuriamo delle versioni
!conda install -y pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

# --- FIX PER L'ERRORE TOKENIZERS ---
# Rimuoviamo "==4.12.5" da transformers per lasciare che pip trovi una versione compatibile con Python 3.11
# Rimuoviamo anche il blocco su timm se dovesse dare problemi, ma proviamo a mantenerlo per ora.
print("Installazione dipendenze Python...")
!pip install timm==0.4.12 transformers decord pandas h5py ftfy regex tqdm opencv-python

# Patch per il path
import sys
if "/content/EgoVLP" not in sys.path:
    sys.path.append("/content/EgoVLP")

✨🍰✨ Everything looks OK!
Cloning into 'EgoVLP'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 268 (delta 85), reused 100 (delta 47), pack-reused 109 (from 1)
Receiving objects: 100% (268/268), 2.00 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/EgoVLP
Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.3
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       7_kmp_llvm           8 KB  conda-forge
    aom-3.6.1                  |       h59595ed_0         2.6 MB  conda-forge
    blas-2.116                 |              mkl          

In [2]:
# 3. Preparazione Dati e Download Pesi Automatico
from google.colab import drive
import os
import shutil
import gdown # Libreria per scaricare da Google Drive

drive.mount('/content/drive')

# --- CONFIGURAZIONE PERCORSI ---
# Dove si trova lo zip dei tuoi video su Drive (questo deve essere corretto!)
PATH_ZIP_VIDEO = "/content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized_3.zip"

# Dove salvare lo zip finale delle features
OUTPUT_ZIP_PATH = "/content/drive/MyDrive/MistakeDetection/egovlp_3.zip"
# -------------------------------

# 1. Setup Cartelle
if not os.path.exists("pretrained"):
    os.makedirs("pretrained")
if not os.path.exists("data/videos"):
    os.makedirs("data/videos")
if not os.path.exists("data/features"):
    os.makedirs("data/features")

# 2. Download Automatico dei Pesi (EgoVLP_PT_BEST.pth)
weights_path = "pretrained/model.pth"
if not os.path.exists(weights_path):
    print("⬇️ Download dei pesi EgoVLP_PT_BEST in corso...")
    # ID del file ufficiale dal GitHub di EgoVLP
    file_id = '1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7'
    url = f'https://drive.google.com/uc?id={file_id}'

    try:
        gdown.download(url, weights_path, quiet=False)
        print("✅ Pesi scaricati correttamente in pretrained/model.pth")
    except Exception as e:
        print(f"❌ Errore nel download: {e}")
        print("Prova a scaricarli manualmente da: https://drive.google.com/file/d/1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7/view")
else:
    print("✅ Pesi già presenti.")

# 3. Estrazione Video
# Controlla se abbiamo già estratto per non rifarlo ogni volta
if len(os.listdir("data/videos")) < 5: # Se la cartella è quasi vuota
    if os.path.exists(PATH_ZIP_VIDEO):
        print(f"📂 Estrazione video da {PATH_ZIP_VIDEO}...")
        !unzip -q "{PATH_ZIP_VIDEO}" -d "data/videos"
        print("✅ Video estratti.")
    else:
        print(f"❌ ERRORE: Non trovo il file zip dei video in: {PATH_ZIP_VIDEO}")
        print("Verifica il percorso nel tuo Google Drive!")
else:
    print("✅ Video già presenti in locale.")

Mounted at /content/drive
⬇️ Download dei pesi EgoVLP_PT_BEST in corso...


Downloading...
From (original): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7
From (redirected): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7&confirm=t&uuid=37db45f9-ac8d-4f83-ad48-8f864c41d1b4
To: /content/EgoVLP/pretrained/model.pth
100%|██████████| 2.17G/2.17G [00:29<00:00, 73.4MB/s]


✅ Pesi scaricati correttamente in pretrained/model.pth
📂 Estrazione video da /content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized_3.zip...
✅ Video estratti.


In [3]:
import os

# Crea la cartella se non esiste
if not os.path.exists("pretrained"):
    os.makedirs("pretrained")

# Scarica il file dei pesi ImageNet richiesto hardcoded nel codice
print("⬇️ Download backbone ViT (richiesto dal codice originale)...")
!wget -nc https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth -O pretrained/jx_vit_base_p16_224-80ecf9dd.pth
print("✅ Backbone scaricato.")

⬇️ Download backbone ViT (richiesto dal codice originale)...
--2025-12-20 09:48:06--  https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth [following]
--2025-12-20 09:48:07--  https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/168799526/65360900-1a09-11eb-8b86-f0a014a6f156?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-20T10%3A28%3A54Z&rscd=attachment%3B+filename%3Djx_vit_base_p16_224-80

In [10]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Successfully uninstalled torchvision-0.20.1
Found existing installation: torchaudio 2.5.1
Uninstalling torchaudio-2.5.1:
  Successfully uninstalled torchaudio-2.5.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 114.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 189.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 194.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [11]:
%%writefile extract_custom.py
import torch
import torch.nn as nn
import glob
import os
import numpy as np
import cv2
import argparse
import time
import queue
import threading
from concurrent.futures import ThreadPoolExecutor
from model.model import FrozenInTime
from utils.util import state_dict_data_parallel_fix

# --- CONFIGURAZIONE PARALLELISMO ---
# Numero di video da leggere contemporaneamente (dipende dai core CPU)
# Su Colab standard 2-3 è ottimale. Su Pro anche 4-6.
NUM_WORKERS = 3
QUEUE_SIZE_PER_WORKER = 2
GLOBAL_QUEUE_SIZE = NUM_WORKERS * QUEUE_SIZE_PER_WORKER

# Parametri Normalizzazione (B, T, C, H, W)
MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1, 1, 3, 1, 1)
STD = torch.tensor([0.229, 0.224, 0.225]).view(1, 1, 3, 1, 1)

def load_model(ckpt_path, device):
    print(f"Costruzione modello FrozenInTime su {device}...")
    model = FrozenInTime(
        video_params={
            "model": "SpaceTimeTransformer",
            "arch_config": "base_patch16_224",
            "num_frames": 16,
            "pretrained": True,
            "time_init": "zeros"
        },
        text_params={"model": "bert-base-uncased", "pretrained": True, "input": "text"},
        projection_dim=256,
        load_checkpoint=None,
        projection='minimal',
        load_temporal_fix='zeros'
    )
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint
    new_state_dict = state_dict_data_parallel_fix(state_dict, model.state_dict())
    model.load_state_dict(new_state_dict, strict=False)
    model = model.to(device)
    model.eval()
    return model

def preprocess_on_gpu(batch_uint8):
    """Normalizza batch uint8 [B, T, H, W, C] -> float32 [B, T, C, H, W]"""
    batch = batch_uint8.float() / 255.0
    batch = batch.permute(0, 1, 4, 2, 3).contiguous()
    batch = (batch - MEAN) / STD
    return batch

def video_worker(vpath, clip_len, batch_size, global_queue):
    """
    Worker che processa un singolo video e invia i batch alla coda globale.
    Invia tuple: (vpath, batch_tensor, is_last_batch)
    """
    try:
        cap = cv2.VideoCapture(vpath)
        if not cap.isOpened():
            # Segnala fine con errore
            global_queue.put((vpath, None, True))
            return

        buffer = []
        current_clip = []

        # Pre-calcolo crop 224x224
        resize_dim = 256
        crop_dim = 224
        start_y = (resize_dim - crop_dim) // 2
        start_x = (resize_dim - crop_dim) // 2

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # CPU Preprocessing
            frame = cv2.resize(frame, (resize_dim, resize_dim), interpolation=cv2.INTER_LINEAR)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame[start_y:start_y+crop_dim, start_x:start_x+crop_dim]

            current_clip.append(frame)

            if len(current_clip) == clip_len:
                buffer.append(np.stack(current_clip))
                current_clip = []

                if len(buffer) >= batch_size:
                    batch_np = np.stack(buffer)
                    batch_tensor = torch.from_numpy(batch_np)
                    # Mette in coda (blocca se piena per non saturare RAM)
                    global_queue.put((vpath, batch_tensor, False))
                    buffer = []

        # Residui Clip
        if len(current_clip) > 0:
            last_frame = current_clip[-1]
            pad_needed = clip_len - len(current_clip)
            current_clip.extend([last_frame] * pad_needed)
            buffer.append(np.stack(current_clip))

        # Residui Batch
        if len(buffer) > 0:
            batch_np = np.stack(buffer)
            batch_tensor = torch.from_numpy(batch_np)
            global_queue.put((vpath, batch_tensor, False))

        cap.release()
        # Segnale di fine video
        global_queue.put((vpath, None, True))

    except Exception as e:
        print(f"Error reading {vpath}: {e}")
        global_queue.put((vpath, None, True))

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--video_dir', type=str, required=True)
    parser.add_argument('--output_dir', type=str, required=True)
    parser.add_argument('--weights', type=str, required=True)
    parser.add_argument('--batch_size', type=int, default=96)
    args = parser.parse_args()

    if torch.cuda.is_available():
        device = 'cuda'
        print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
        print(f"⚡ MULTI-VIDEO MODE: {NUM_WORKERS} Workers Paralleli | Batch {args.batch_size}")
    else:
        device = 'cpu'
        print("⚠️ CUDA non disponibile.")

    global MEAN, STD
    MEAN = MEAN.to(device)
    STD = STD.to(device)

    model = load_model(args.weights, device)

    # Lista video da processare
    all_videos = []
    for ext in ['*.mp4', '*.avi', '*.MOV']:
        all_videos.extend(glob.glob(os.path.join(args.video_dir, "**", ext), recursive=True))

    # Filtra quelli già fatti
    videos_to_process = []
    for vpath in all_videos:
        vname = os.path.basename(vpath)
        save_name = f"{vname}_1s_1s.npz"
        save_path = os.path.join(args.output_dir, save_name)
        if not os.path.exists(save_path):
            videos_to_process.append(vpath)

    print(f"Trovati {len(all_videos)} video totali. Da elaborare: {len(videos_to_process)}")

    if not videos_to_process:
        print("Tutti i video completati!")
        return

    # Strutture dati condivise
    global_queue = queue.Queue(maxsize=GLOBAL_QUEUE_SIZE)
    results_buffer = {} # {vpath: [list_of_features]}
    active_videos = set()
    completed_videos = 0
    start_times = {}

    # Executor per i worker
    executor = ThreadPoolExecutor(max_workers=NUM_WORKERS)

    # Funzione helper per avviare un worker
    video_iterator = iter(videos_to_process)

    def start_next_worker():
        try:
            vpath = next(video_iterator)
            active_videos.add(vpath)
            results_buffer[vpath] = []
            start_times[vpath] = time.time()
            executor.submit(video_worker, vpath, 16, args.batch_size, global_queue)
            print(f"▶️ Start: {os.path.basename(vpath)}")
            return True
        except StopIteration:
            return False

    # Avvia i primi N worker
    for _ in range(NUM_WORKERS):
        start_next_worker()

    # MAIN LOOP (GPU Consumer)
    while active_videos:
        try:
            # Preleva un batch dalla coda (da qualsiasi video)
            vpath, batch_cpu, is_last = global_queue.get(timeout=60) # Timeout per evitare deadlock

            if is_last:
                # Video finito: Salva e chiudi
                features = np.concatenate(results_buffer[vpath], axis=0)

                vname = os.path.basename(vpath)
                save_name = f"{vname}_1s_1s.npz"
                save_path = os.path.join(args.output_dir, save_name)
                np.savez_compressed(save_path, features=features.astype(np.float32))

                elapsed = time.time() - start_times[vpath]
                fps = (features.shape[0] * 16) / elapsed
                completed_videos += 1
                print(f"✅ Saved: {vname} ({elapsed:.1f}s - {fps:.0f} fps) | Progress: {completed_videos}/{len(videos_to_process)}")

                # Pulizia
                del results_buffer[vpath]
                del start_times[vpath]
                active_videos.remove(vpath)

                # Avvia prossimo video
                start_next_worker()
                global_queue.task_done()
                continue

            if batch_cpu is not None:
                # Inferenza GPU
                batch_gpu = batch_cpu.to(device, non_blocking=True)
                input_batch = preprocess_on_gpu(batch_gpu)

                with torch.amp.autocast('cuda'):
                    with torch.no_grad():
                        v_embed = model.compute_video(input_batch)

                # Salva risultato nel buffer specifico del video
                results_buffer[vpath].append(v_embed.cpu().numpy().astype(np.float16))

            global_queue.task_done()

        except queue.Empty:
            print("⚠️ Warning: GPU Queue empty for 60s. Workers might be stuck.")
            break
        except Exception as e:
            print(f"❌ Critical Error in Main Loop: {e}")
            break

    executor.shutdown()
    print("🎉 Tutti i video elaborati.")

if __name__ == "__main__":
    main()

Overwriting extract_custom.py


In [ ]:
!pip install einops
!pip install humanize psutil
!pip install ffmpeg-python
!pip install av dominate scikit-learn tensorboardx

# Crea cartella output
!mkdir -p data/features

# Esegui lo script
# Assicurati che VIDEO_ROOT punti alla cartella dove ci sono effettivamente i file .mp4
!python extract_custom.py \
  --video_dir "data/videos/captain_cook_4d_gopro_resized_3" \
  --output_dir "data/features" \
  --weights "pretrained/model.pth" \
  --batch_size 64

🚀 GPU: Tesla T4
⚡ MULTI-VIDEO MODE: 3 Workers Paralleli | Batch 64
Costruzione modello FrozenInTime su cuda...
/content/EgoVLP/model/model.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [6]:
print("Creazione archivio zip...")
!zip -r -q egovlp_features.zip data/features

print(f"Upload su Drive: {OUTPUT_ZIP_PATH}")
shutil.copy("egovlp_features.zip", OUTPUT_ZIP_PATH)
print("✅ Completato!")

Creazione archivio zip...
Upload su Drive: /content/drive/MyDrive/MistakeDetection/egovlp_3.zip
✅ Completato!
